In [1]:
pwd

'/Users/telekenesis/Husky Gdrive/Coursework/7. SML_EE/Project/Face-Detection-PyTorch'

In [3]:
from platform import python_version

print(python_version())

3.6.12


In [4]:
import cv2


ModuleNotFoundError: No module named 'cv2'